In [5]:
#搭建串联的网络结构
import torch.nn as nn
import torch.nn.functional as F


class VGGbase(nn.Module):
    def __init__(self): #初始化
        super(VGGbase, self).__init__()
        #3 * 28 * 28 (crop-->32, 28)
        #定义第一组卷积
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.max_pooling = nn.MaxPool2d(kernel_size=2, stride=2)

        #14 *14
        #定义第二组卷积
        self.conv2_1 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        self.conv2_2 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        self.max_pooling2 = nn.MaxPool2d(kernel_size=2, stride=2)

        #7 * 7
        #定义第三组卷积
        self.conv3_1 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )
        self.conv3_2 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )
        self.max_pooling3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        #不加padding会导致输出变成3*3,变少。加上之后输出变成4*4

        #4 * 4
        #定义第四组卷积
        self.conv4_1 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        self.conv4_2 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        self.max_pooling4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        #这样就变成了2*2 下面定义fc层
        #batchsize * 512 * 2 * 2 --> batchsize *(512 * 4)
        self.fc = nn.Linear(512 *4, 10)


    def forward(self, x):
        batchsize = x.size(0)
        out = self.conv1(x)
        out = self.max_pooling(out)
        #第二层串联
        out = self.conv2_1(out)
        out = self.conv2_2(out)
        out = self.max_pooling2(out)
        #第三层串联
        out = self.conv3_1(out)
        out = self.conv3_2(out)
        out = self.max_pooling3(out)
        #第四层串联
        out = self.conv4_1(out)
        out = self.conv4_2(out)
        out = self.max_pooling4(out)
        #fc层之前先展平
        out = out.view(batchsize, -1)#-1是自动识别
        #fc层
        #batchsize * C * h * w --> batchsize * n
        out = self.fc(out) #batchsize * 10
        
        out = F.log_softmax(out, dim=1)
        
        return out

def VGGNet():
    return VGGbase()



